In [ ]:
%load_ext autoreload
%autoreload 2
from src.general_methods.visualizer import Visualizer
from src.deep.data_analyzer import DataAnalyzer
from tqdm.notebook import tqdm

In [ ]:
root_dir = f'../../data/datasets/qam1024_160x20'
data_analyzer = DataAnalyzer(root_dir, _tqdm=tqdm, is_box_plot=True)
data_analyzer.plot_single_sample(0.001, data_id=0, is_save=False)

In [5]:
mu = 0.001
n = 20
ber_vec = data_analyzer.calc_ber_for_sub_folder(mu=mu, n=n, _tqdm=tqdm)

  0%|          | 0/20 [00:00<?, ?it/s]

AssertionError: oh no, the outcome is not 5120, but 0

In [ ]:
Visualizer.plot_bers_boxplot(us=[mu], bers_vecs=ber_vec, output_path=None )
Visualizer.my_plot(range(n), ber_vec, xlabel='i', ylabel='BER', function='step')

In [ ]:
data_analyzer.plot_full_ber_graph(n=20, is_save=False)